***Imports***

In [2]:
import pandas as pd

***Loading Datasets***

In [3]:
# Data for Content Filtering
games_content = pd.read_csv('../data/raw-data/content-filtering/games.csv')
genres = pd.read_csv('../data/raw-data/content-filtering/genres.csv')
platforms = pd.read_csv('../data/raw-data/content-filtering/platforms.csv')
scores = pd.read_csv('../data/raw-data/content-filtering/scores.csv')
developers = pd.read_csv('../data/raw-data/content-filtering/developers.csv')

# Data for Collaborative Filtering
games_collab = pd.read_csv('../data/raw-data/collaborative-filtering/steam-200k.csv',
                            names=['userid', 'name', 'behavior', 'hours', '0']) # creating column names

# **Cleaning Content Filtering Data**

### **Cleaning Genres Dataset**

***Combining Genres together into One Column***

In [4]:
# list of game ids
game_id = genres['id'].unique().tolist()
new_genres = pd.DataFrame()

# combining multiple genres into one column
for id in game_id:
    genre = genres.loc[genres['id'] == id, 'genre'].tolist()
    new_genre = pd.DataFrame({'id' : id, 'genre' : [genre]})
    new_genres = pd.concat([new_genres, new_genre], ignore_index=True)


***Finished Clean Product***

In [5]:
new_genres.head()

,id,genre
0,1000001,[Point-and-Click]
1,1000002,"[Puzzle, Tactical]"
2,1000003,"[Pinball, Strategy]"
3,1000004,[Card & Board Game]
4,1000005,"[Puzzle, Strategy]"


### **Cleaning Platforms Dataset**

***Removing Lesser Known Platforms***

In [6]:
# creating list of platforms that have at least 10000 games
relevant_platforms = platforms['platform'].value_counts()[platforms['platform'].value_counts() > 1000].index.tolist()

# filtering only relevant platforms
filtered_platforms = platforms[platforms['platform'].isin(relevant_platforms)]

***Combining Platforms Together into One Column***

In [7]:
# list of platform ids
platform_id = filtered_platforms['id'].unique().tolist()
new_platforms = pd.DataFrame()

# combining multiple platforms into one column
for id in platform_id:
    platform = filtered_platforms.loc[filtered_platforms['id'] == id, 'platform'].tolist()
    new_platform = pd.DataFrame({'id' : id, 'platform' : [platform]})
    new_platforms = pd.concat([new_platforms, new_platform], ignore_index=True)

***Removing Punctuation***

In [8]:
def remove_punctuation(platforms): # Removes punctuation from text
    for index in range(len(platforms)):
        text = platforms[index].lower() # lowers all letters
        alphabet = 'abcdefghijklmnopqrstuvwxyz '
        text = "".join([char for char in text if char in alphabet])
        platforms[index] = text
    return platforms

# applying function to data
new_platforms['platform'] = new_platforms['platform'].apply(lambda x: remove_punctuation(x))

***Final Cleaned Product***

In [9]:
new_platforms.tail()

,id,platform
133569,1172414,"[windows pc, playstation , xbox one, playstati..."
133570,1172418,[windows pc]
133571,1172495,[windows pc]
133572,1172511,[windows pc]
133573,1172512,"[android, ios]"


### **Cleaning Scores Dataset**

***Combining the Score and Amount into a Mean***

In [10]:
# creating list of scores
score_id = scores['id'].unique().tolist()
new_scores = pd.DataFrame()

# creating the mean using score and amount
for id in score_id:
    score = scores[scores['id'] == id]['score'].tolist()
    amount = scores[scores['id'] == id]['amount'].tolist()
    addition = 0
    for index in range(len(score)): # adds every single value together
        add = score[index] * amount[index]
        addition += add
    amount_total = 0
    for number in amount: # getting the total amount of reviews
        amount_total += number
    if amount_total > 0: # as long there are some sort of reviews
        mean = addition / amount_total
    else:
        mean = 0
    new_score = pd.DataFrame({'id' : id, 'score' : mean}, index=[0])
    new_scores = pd.concat([new_score, new_scores], ignore_index=True)

***Removing Scores that have a 0***

In [11]:
new_scores = new_scores[new_scores['score'] > 0]

***Final Cleaned Product***

In [12]:
new_scores.head()

,id,score
0,1172512,3.50
7,1172505,2.50
16,1172496,2.00
18,1172494,3.50
20,1172492,3.25


### **Cleaning Developers Dataset**

***Removing Nulls***

In [13]:
developers.dropna(inplace=True)

***Combining Developers Together into one Column***

In [14]:
# list of developers ids
developer_id = developers['id'].unique().tolist()
new_developers = pd.DataFrame()

# combining multiple developers into one column
for id in developer_id:
    developer = developers.loc[developers['id'] == id, 'developer'].tolist()
    new_developer = pd.DataFrame({'id' : id, 'developer' : [developer]})
    new_developers = pd.concat([new_developers, new_developer], ignore_index=True)

***Final Cleaned Product***

In [15]:
new_developers.head()

,id,developer
0,1000002,[Josef Kates]
1,1000004,[Christopher Strachey]
2,1000005,"[Alexander Shafto ""Sandy"" Douglas, University ..."
3,1000007,[William Higinbotham]
4,1000009,"[Steve Russel, Computer Recreations, Inc.]"


### **Joining All Datasets**

In [16]:
games_content = pd.merge(games_content, new_genres, on='id', how='left')
games_content = pd.merge(games_content, new_platforms, on='id', how='left')
games_content = pd.merge(games_content, new_scores, on='id', how='left')
games_content = pd.merge(games_content, new_developers, on='id', how='left')

In [17]:
games_content.head()

,id,name,date,rating,reviews,plays,playing,backlogs,wishlists,description,genre,platform,score,developer
0,1000001,Cathode Ray Tube Amusement Device,1947-12-31,3.5,65.0,117.0,1.0,28.0,56.0,The cathode ray tube amusement device is the e...,[Point-and-Click],NaN,3.529070,NaN
1,1000002,Bertie the Brain,1950-08-25,2.5,11.0,24.0,0.0,6.0,12.0,Currently considered the first videogame in hi...,"[Puzzle, Tactical]",[arcade],2.535714,[Josef Kates]
2,1000003,Nim,1951-12-31,1.8,2.0,11.0,0.0,2.0,6.0,The Nimrod was a special purpose computer that...,"[Pinball, Strategy]",NaN,1.750000,NaN
3,1000004,Draughts,1952-08-31,2.4,3.0,17.0,0.0,3.0,7.0,A game of draughts (a.k.a. checkers) written f...,[Card & Board Game],NaN,2.388889,[Christopher Strachey]
4,1000005,OXO,1952-12-31,3.1,14.0,52.0,1.0,12.0,13.0,OXO was a computer game developed by Alexander...,"[Puzzle, Strategy]",[windows pc],3.066667,"[Alexander Shafto ""Sandy"" Douglas, University ..."


### **Cleaning Games Dataset**

***Filtering only relevant columns***

In [18]:
relevant_columns = ['id', 'name', 'date', 'plays', 'description', 'genre', 'platform',
                    'score', 'developer']

games_content = games_content[relevant_columns]

***Removing Games with any Nulls***

In [19]:
games_content = games_content.dropna()

***Cleaning Date to only Year***

In [20]:
def change_date(date):
    return int(date[:4])

games_content['date'] = games_content['date'].apply(change_date)

***Only Using Games that have more than 100 players***

In [21]:
games_content = games_content[games_content['plays'] > 100]

***Removing Punctuation in name***

In [22]:
def remove_punctuation(text): # Removes punctuation from text
    text = text.lower()
    alphabet = 'abcdefghijklmnopqrstuvwxyz 1234567890'
    text = "".join([char for char in text if char in alphabet])
    return text

# applying function to data
games_content['name'] = games_content['name'].apply(lambda x: remove_punctuation(x))

***Removing Duplicate Games***

In [23]:
games_content = games_content.drop_duplicates(subset='name')

### **Final Games Content Dataset**

In [24]:
games_content.head()

,id,name,date,plays,description,genre,platform,score,developer
41,1000042,pong,1972,4053.0,Pong is a tennis game with simple two-dimensio...,"[Arcade, Simulator, Sport]",[arcade],2.985992,"[Fairchild Semiconductor, Atari, Inc.]"
149,1000150,breakout,1976,558.0,The objective of the game is to destroy a mult...,[Arcade],[arcade],3.053628,"[Namco, Atari]"
201,1000202,zork,1977,281.0,Zork is one of the earliest interactive fictio...,[Adventure],"[mac, atari stste, amiga, pc dos, commodore c,...",3.164062,"[Commodore Electronics Ltd., Infocom]"
232,1000233,airsea battle,1977,123.0,There are six basic types of game available in...,[Shooter],[windows pc],2.300000,"[Atari, Inc., Atari]"
299,1000300,space invaders,1978,2811.0,Space Invaders is an arcade video game develop...,"[Arcade, Shooter]","[ios, arcade]",3.104677,"[Sharp, Taito Corporation]"


In [25]:
games_content.shape

(10429, 9)

**Insights:** This is the final cleaned content data. The description column will be later cleaned later for specific NLP tasks. In the end we are left with `10429` games.

***Saving Dataset***

In [26]:
games_content.to_csv('../data/cleaned-data/games_content.csv')

# **Cleaning Collaborative Filtering Data**

***Filtering Whether a User has either Played or Purchased***

In [74]:
# get ids for users 
games_id = games_collab['userid'].unique().tolist()
new_games_collab = pd.DataFrame()


for id in games_id:
    games_list = games_collab[games_collab['userid'] == id]['name'].unique().tolist()
    for game in games_list:
        # getting behaviours for the user's game
        behaviors = games_collab[(games_collab['userid'] == id) & (games_collab['name'] == game)]['behavior'].tolist()
        if 'play' in behaviors: # if played before apply play
            user_game = pd.DataFrame({'userid' : id, 'name' : game, 'behavior' : 'play'}, index=[0])
        else: # else just purchased
            user_game = pd.DataFrame({'userid' : id, 'name' : game, 'behavior' : 'purchase'}, index=[0])
        new_games_collab = pd.concat([new_games_collab, user_game], ignore_index=True)
    

***Filtering Users on how many games they've played***

In [81]:
# filters users if they have purchased/played at least 2 games
users = new_games_collab['userid'].value_counts()[new_games_collab['userid'].value_counts() > 1].index.tolist()

new_games_collab = new_games_collab[new_games_collab['userid'].isin(users)]

***Removing Punctuation***

In [82]:
def remove_punctuation(text): # Removes punctuation from text
    text = text.lower()
    alphabet = 'abcdefghijklmnopqrstuvwxyz 1234567890'
    text = "".join([char for char in text if char in alphabet])
    return text

# applying function to data
new_games_collab['name'] = new_games_collab['name'].apply(lambda x: remove_punctuation(x))

/var/folders/m2/l617y1qd3w54klyxl13frw5h0000gn/T/ipykernel_12121/3352423872.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_games_collab['name'] = new_games_collab['name'].apply(lambda x: remove_punctuation(x))


### **Final Games Collaborative Dataset**

In [83]:
new_games_collab.head()

,userid,name,behavior
0,151603712,the elder scrolls v skyrim,play
1,151603712,fallout 4,play
2,151603712,spore,play
3,151603712,fallout new vegas,play
4,151603712,left 4 dead 2,play


In [85]:
new_games_collab.shape

(123104, 3)

In [88]:
len(new_games_collab['userid'].unique())

6693

**Insights:** In the final dataset we have **123104** games that players have either played or purchased. With **6693** different users who have played/purchased at least 2 games.

***Saving Dataset***

In [89]:
new_games_collab.to_csv('../data/cleaned-data/games_collab.csv')